# ETL - Extração, Transformação e Carga de Dados de E-commerce

Este notebook implementa um pipeline completo de ETL (Extração, Transformação e Carga) para dados de e-commerce da Olist, preparando-os para visualização no Power BI.

## Importação de Bibliotecas

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import sys
import warnings

# Ignorando avisos para melhor visualização
warnings.filterwarnings('ignore')

# Adicionando diretório src ao path para importar módulos personalizados
sys.path.append('../src')
import etl_functions as etl

%matplotlib inline

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
pd.set_option('display.max_columns', None)

## 1. Extração de Dados

Nesta seção, extraímos dados de diferentes fontes. Em um cenário real, isso poderia incluir bancos de dados, APIs, arquivos CSV, Excel, etc. Para este exemplo, usaremos dados de e-commerce da Olist disponíveis no Kaggle.

In [ ]:
# Definindo função para criar dados de exemplo se os arquivos não existirem
def create_sample_data():
    """Cria dados de exemplo para demonstração do ETL"""
    print("Criando dados de exemplo para demonstração...")
    
    # Criando diretório se não existir
    os.makedirs('../data/raw', exist_ok=True)
    
    # Dados de exemplo - Clientes
    customers_data = {
        'customer_id': [f'cust_{i}' for i in range(1, 101)],
        'customer_unique_id': [f'uniq_{i}' for i in range(1, 101)],
        'customer_zip_code_prefix': np.random.randint(10000, 99999, 100),
        'customer_city': np.random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Porto Alegre', 'Brasília'], 100),
        'customer_state': np.random.choice(['SP', 'RJ', 'MG', 'RS', 'DF'], 100)
    }
    pd.DataFrame(customers_data).to_csv('../data/raw/olist_customers_dataset.csv', index=False)
    
    # Dados de exemplo - Pedidos
    orders_data = {
        'order_id': [f'order_{i}' for i in range(1, 201)],
        'customer_id': np.random.choice([f'cust_{i}' for i in range(1, 101)], 200),
        'order_status': np.random.choice(['delivered', 'shipped', 'processing', 'canceled'], 200, p=[0.7, 0.1, 0.1, 0.1]),
        'order_purchase_timestamp': pd.date_range(start='2022-01-01', end='2022-12-31', periods=200).astype(str),
        'order_approved_at': pd.date_range(start='2022-01-01', end='2022-12-31', periods=200).astype(str),
        'order_delivered_carrier_date': pd.date_range(start='2022-01-02', end='2023-01-01', periods=200).astype(str),
        'order_delivered_customer_date': pd.date_range(start='2022-01-05', end='2023-01-05', periods=200).astype(str),
        'order_estimated_delivery_date': pd.date_range(start='2022-01-10', end='2023-01-10', periods=200).astype(str)
    }
    pd.DataFrame(orders_data).to_csv('../data/raw/olist_orders_dataset.csv', index=False)
    
    # Dados de exemplo - Produtos
    products_data = {
        'product_id': [f'prod_{i}' for i in range(1, 151)],
        'product_category_name': np.random.choice(['electronics', 'furniture', 'toys', 'books', 'clothing'], 150),
        'product_name_length': np.random.randint(10, 100, 150),
        'product_description_length': np.random.randint(100, 1000, 150),
        'product_photos_qty': np.random.randint(1, 10, 150),
        'product_weight_g': np.random.randint(100, 10000, 150),
        'product_length_cm': np.random.randint(10, 100, 150),
        'product_height_cm': np.random.randint(5, 50, 150),
        'product_width_cm': np.random.randint(5, 50, 150)
    }
    pd.DataFrame(products_data).to_csv('../data/raw/olist_products_dataset.csv', index=False)
    
    # Dados de exemplo - Itens de Pedido
    order_items_data = {
        'order_id': np.random.choice([f'order_{i}' for i in range(1, 201)], 300),
        'order_item_id': np.random.randint(1, 5, 300),
        'product_id': np.random.choice([f'prod_{i}' for i in range(1, 151)], 300),
        'seller_id': np.random.choice([f'seller_{i}' for i in range(1, 51)], 300),
        'shipping_limit_date': pd.date_range(start='2022-01-01', end='2022-12-31', periods=300).astype(str),
        'price': np.random.uniform(10, 1000, 300).round(2),
        'freight_value': np.random.uniform(5, 100, 300).round(2)
    }
    pd.DataFrame(order_items_data).to_csv('../data/raw/olist_order_items_dataset.csv', index=False)
    
    # Dados de exemplo - Vendedores
    sellers_data = {
        'seller_id': [f'seller_{i}' for i in range(1, 51)],
        'seller_zip_code_prefix': np.random.randint(10000, 99999, 50),
        'seller_city': np.random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Curitiba', 'Salvador'], 50),
        'seller_state': np.random.choice(['SP', 'RJ', 'MG', 'PR', 'BA'], 50)
    }
    pd.DataFrame(sellers_data).to_csv('../data/raw/olist_sellers_dataset.csv', index=False)
    
    # Dados de exemplo - Avaliações
    reviews_data = {
        'review_id': [f'review_{i}' for i in range(1, 201)],
        'order_id': [f'order_{i}' for i in range(1, 201)],
        'review_score': np.random.randint(1, 6, 200),
        'review_comment_title': [f'Title {i}' if i % 3 == 0 else None for i in range(1, 201)],
        'review_comment_message': [f'Message {i}' if i % 2 == 0 else None for i in range(1, 201)],
        'review_creation_date': pd.date_range(start='2022-01-01', end='2022-12-31', periods=200).astype(str),
        'review_answer_timestamp': pd.date_range(start='2022-01-02', end='2023-01-01', periods=200).astype(str)
    }
    pd.DataFrame(reviews_data).to_csv('../data/raw/olist_order_reviews_dataset.csv', index=False)
    
    # Dados de exemplo - Categorias em Português/Inglês
    category_translation_data = {
        'product_category_name': ['electronics', 'furniture', 'toys', 'books', 'clothing'],
        'product_category_name_english': ['electronics', 'furniture', 'toys', 'books', 'clothing']
    }
    pd.DataFrame(category_translation_data).to_csv('../data/raw/product_category_name_translation.csv', index=False)
    
    print("Dados de exemplo criados com sucesso!")

# Verificando se os arquivos existem, caso contrário, criando dados de exemplo
required_files = [
    '../data/raw/olist_customers_dataset.csv',
    '../data/raw/olist_orders_dataset.csv',
    '../data/raw/olist_products_dataset.csv',
    '../data/raw/olist_order_items_dataset.csv',
    '../data/raw/olist_sellers_dataset.csv',
    '../data/raw/olist_order_reviews_dataset.csv',
    '../data/raw/product_category_name_translation.csv'
]

if not all(os.path.exists(file) for file in required_files):
    create_sample_data()

In [ ]:
# Extraindo dados dos arquivos CSV
print("Extraindo dados...")

# Usando a função de extração do módulo etl_functions
raw_data = etl.extract_data('../data/raw/')

# Verificando os dataframes extraídos
for name, df in raw_data.items():
    print(f"{name}: {df.shape[0]} linhas, {df.shape[1]} colunas")

## 2. Transformação de Dados

Nesta seção, realizamos várias transformações nos dados brutos para prepará-los para análise e visualização.

In [ ]:
# Convertendo colunas de data para datetime
print("Transformando dados...")

# Usando a função de transformação do módulo etl_functions
transformed_data = etl.transform_data(raw_data)

# Verificando os dataframes transformados
for name, df in transformed_data.items():
    print(f"{name}: {df.shape[0]} linhas, {df.shape[1]} colunas")

In [ ]:
# Criando tabelas dimensionais e fato para modelo estrela
print("Criando modelo dimensional...")

# Usando a função de criação de modelo dimensional do módulo etl_functions
dim_tables, fact_table = etl.create_dimensional_model(transformed_data)

# Verificando as tabelas dimensionais
print("\nTabelas Dimensionais:")
for name, df in dim_tables.items():
    print(f"{name}: {df.shape[0]} linhas, {df.shape[1]} colunas")

# Verificando a tabela fato
print(f"\nTabela Fato: {fact_table.shape[0]} linhas, {fact_table.shape[1]} colunas")

### Visualizando Exemplos das Tabelas Dimensionais

In [ ]:
# Visualizando exemplos das tabelas dimensionais
for name, df in dim_tables.items():
    print(f"\n{name} (primeiras 5 linhas):")
    display(df.head())

### Visualizando Exemplo da Tabela Fato

In [ ]:
# Visualizando exemplo da tabela fato
print("\nTabela Fato (primeiras 5 linhas):")
display(fact_table.head())

## 3. Criação de Tabelas Agregadas para Análise

Nesta seção, criamos tabelas agregadas que serão úteis para visualizações e dashboards.

In [ ]:
# Criando tabelas agregadas para análise
print("Criando tabelas agregadas...")

# Usando a função de criação de tabelas agregadas do módulo etl_functions
agg_tables = etl.create_aggregated_tables(fact_table, dim_tables)

# Verificando as tabelas agregadas
for name, df in agg_tables.items():
    print(f"{name}: {df.shape[0]} linhas, {df.shape[1]} colunas")

### Visualizando Exemplos das Tabelas Agregadas

In [ ]:
# Visualizando exemplos das tabelas agregadas
for name, df in agg_tables.items():
    print(f"\n{name} (primeiras 5 linhas):")
    display(df.head())

## 4. Carga de Dados

Nesta seção, salvamos os dados transformados e agregados em formatos adequados para uso no Power BI.

In [ ]:
# Salvando dados transformados
print("Salvando dados transformados...")

# Criando diretório para dados transformados se não existir
os.makedirs('../data/transformed', exist_ok=True)

# Salvando tabelas dimensionais
for name, df in dim_tables.items():
    df.to_csv(f'../data/transformed/dim_{name}.csv', index=False)
    print(f"Tabela dimensional '{name}' salva com sucesso.")

# Salvando tabela fato
fact_table.to_csv('../data/transformed/fact_sales.csv', index=False)
print("Tabela fato 'sales' salva com sucesso.")

# Salvando tabelas agregadas
for name, df in agg_tables.items():
    df.to_csv(f'../data/transformed/agg_{name}.csv', index=False)
    print(f"Tabela agregada '{name}' salva com sucesso.")

In [ ]:
# Salvando dados em formato parquet para melhor desempenho no Power BI
print("\nSalvando dados em formato parquet...")

# Salvando tabelas dimensionais em parquet
for name, df in dim_tables.items():
    df.to_parquet(f'../data/transformed/dim_{name}.parquet')
    print(f"Tabela dimensional '{name}' salva em parquet com sucesso.")

# Salvando tabela fato em parquet
fact_table.to_parquet('../data/transformed/fact_sales.parquet')
print("Tabela fato 'sales' salva em parquet com sucesso.")

# Salvando tabelas agregadas em parquet
for name, df in agg_tables.items():
    df.to_parquet(f'../data/transformed/agg_{name}.parquet')
    print(f"Tabela agregada '{name}' salva em parquet com sucesso.")

## 5. Verificação dos Dados Transformados

Nesta seção, realizamos algumas verificações para garantir que os dados foram transformados corretamente.

In [ ]:
# Verificando integridade dos dados
print("Verificando integridade dos dados...")

# Verificando se todas as chaves estrangeiras na tabela fato existem nas tabelas dimensionais
for dim_name, dim_df in dim_tables.items():
    if f'{dim_name}_id' in fact_table.columns:
        fk_values = fact_table[f'{dim_name}_id'].unique()
        pk_values = dim_df['id'].unique()
        missing_keys = set(fk_values) - set(pk_values)
        if missing_keys:
            print(f"ERRO: Chaves estrangeiras ausentes na dimensão {dim_name}: {missing_keys}")
        else:
            print(f"OK: Todas as chaves estrangeiras para a dimensão {dim_name} estão presentes.")

# Verificando valores nulos em colunas importantes
print("\nVerificando valores nulos em colunas importantes da tabela fato:")
null_counts = fact_table[['order_id', 'customer_id', 'product_id', 'seller_id', 'date_id', 'price', 'freight_value']].isnull().sum()
print(null_counts)

# Verificando consistência dos dados agregados
print("\nVerificando consistência dos dados agregados:")
total_sales_fact = fact_table['price'].sum()
total_sales_agg = agg_tables['sales_by_date']['total_sales'].sum()
print(f"Total de vendas na tabela fato: {total_sales_fact:.2f}")
print(f"Total de vendas na tabela agregada por data: {total_sales_agg:.2f}")
print(f"Diferença: {abs(total_sales_fact - total_sales_agg):.2f}")

## 6. Visualização Prévia para o Dashboard

Nesta seção, criamos algumas visualizações preliminares que servirão de base para o dashboard no Power BI.

In [ ]:
# Criando visualizações preliminares
print("Criando visualizações preliminares...")

# Criando diretório para figuras se não existir
os.makedirs('../reports/dashboard', exist_ok=True)

# Vendas por mês
plt.figure(figsize=(12, 6))
sns.barplot(x='month', y='total_sales', data=agg_tables['sales_by_date'], palette='viridis')
plt.title('Vendas Totais por Mês', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Vendas Totais (R$)', fontsize=12)
plt.xticks(range(12), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.grid(True, alpha=0.3)
plt.savefig('../reports/dashboard/sales_by_month.png', dpi=300, bbox_inches='tight')
plt.show()

# Vendas por categoria
plt.figure(figsize=(14, 8))
category_sales = agg_tables['sales_by_category'].sort_values('total_sales', ascending=False).head(10)
sns.barplot(y='category_name', x='total_sales', data=category_sales, palette='viridis')
plt.title('Top 10 Categorias por Vendas', fontsize=16)
plt.xlabel('Vendas Totais (R$)', fontsize=12)
plt.ylabel('Categoria', fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('../reports/dashboard/sales_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

# Vendas por estado
plt.figure(figsize=(14, 8))
state_sales = agg_tables['sales_by_location'].sort_values('total_sales', ascending=False).head(10)
sns.barplot(y='state', x='total_sales', data=state_sales, palette='viridis')
plt.title('Top 10 Estados por Vendas', fontsize=16)
plt.xlabel('Vendas Totais (R$)', fontsize=12)
plt.ylabel('Estado', fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('../reports/dashboard/sales_by_state.png', dpi=300, bbox_inches='tight')
plt.show()

# Distribuição de avaliações
plt.figure(figsize=(10, 6))
sns.countplot(x='review_score', data=dim_tables['review'], palette='viridis')
plt.title('Distribuição de Avaliações', fontsize=16)
plt.xlabel('Pontuação da Avaliação', fontsize=12)
plt.ylabel('Número de Avaliações', fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('../reports/dashboard/review_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Criação de Imagem de Exemplo do Dashboard

Nesta seção, criamos uma imagem de exemplo do dashboard que será incluída no README do projeto.

In [ ]:
# Criando uma imagem de exemplo do dashboard
print("Criando imagem de exemplo do dashboard...")

# Criando um layout de dashboard simples com as visualizações criadas
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# Vendas por mês
sns.barplot(x='month', y='total_sales', data=agg_tables['sales_by_date'], palette='viridis', ax=axes[0, 0])
axes[0, 0].set_title('Vendas Totais por Mês', fontsize=16)
axes[0, 0].set_xlabel('Mês', fontsize=12)
axes[0, 0].set_ylabel('Vendas Totais (R$)', fontsize=12)
axes[0, 0].set_xticks(range(12))
axes[0, 0].set_xticklabels(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
axes[0, 0].grid(True, alpha=0.3)

# Vendas por categoria
category_sales = agg_tables['sales_by_category'].sort_values('total_sales', ascending=False).head(10)
sns.barplot(y='category_name', x='total_sales', data=category_sales, palette='viridis', ax=axes[0, 1])
axes[0, 1].set_title('Top 10 Categorias por Vendas', fontsize=16)
axes[0, 1].set_xlabel('Vendas Totais (R$)', fontsize=12)
axes[0, 1].set_ylabel('Categoria', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

# Vendas por estado
state_sales = agg_tables['sales_by_location'].sort_values('total_sales', ascending=False).head(10)
sns.barplot(y='state', x='total_sales', data=state_sales, palette='viridis', ax=axes[1, 0])
axes[1, 0].set_title('Top 10 Estados por Vendas', fontsize=16)
axes[1, 0].set_xlabel('Vendas Totais (R$)', fontsize=12)
axes[1, 0].set_ylabel('Estado', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)

# Distribuição de avaliações
sns.countplot(x='review_score', data=dim_tables['review'], palette='viridis', ax=axes[1, 1])
axes[1, 1].set_title('Distribuição de Avaliações', fontsize=16)
axes[1, 1].set_xlabel('Pontuação da Avaliação', fontsize=12)
axes[1, 1].set_ylabel('Número de Avaliações', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)

# Adicionando título geral
fig.suptitle('Dashboard de Vendas E-commerce', fontsize=24, y=0.98)

# Ajustando layout
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Salvando imagem do dashboard
plt.savefig('../reports/dashboard/dashboard_preview.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Instruções para Integração com Power BI

Nesta seção, fornecemos instruções para integrar os dados transformados com o Power BI.

### Passos para Integração com Power BI

1. **Abra o Power BI Desktop**

2. **Importe os dados transformados**:
   - Clique em "Obter Dados" > "Arquivo" > "Pasta"
   - Navegue até a pasta `data/transformed`
   - Selecione os arquivos .parquet (recomendado) ou .csv

3. **Configure as relações entre tabelas**:
   - Vá para a visualização "Modelo"
   - Crie relações entre a tabela fato e as tabelas dimensionais usando as chaves correspondentes:
     - fact_sales.customer_id → dim_customer.id
     - fact_sales.product_id → dim_product.id
     - fact_sales.seller_id → dim_seller.id
     - fact_sales.date_id → dim_date.id
     - fact_sales.order_id → dim_order.id

4. **Crie medidas calculadas**:
   - Clique com o botão direito na tabela fact_sales > "Nova medida"
   - Crie medidas como:
     ```
     Total Vendas = SUM(fact_sales[price])
     Total Frete = SUM(fact_sales[freight_value])
     Ticket Médio = DIVIDE(SUM(fact_sales[price]), DISTINCTCOUNT(fact_sales[order_id]))
     ```

5. **Crie visualizações**:
   - Gráfico de barras para vendas por categoria
   - Gráfico de linhas para tendência de vendas ao longo do tempo
   - Mapa para vendas por estado
   - Cartões para KPIs como total de vendas, número de pedidos, ticket médio
   - Gráfico de pizza para distribuição de avaliações

6. **Adicione segmentações de dados**:
   - Adicione segmentações por período, categoria, estado, etc.

7. **Formate o dashboard**:
   - Organize as visualizações
   - Adicione título e descrições
   - Aplique um tema consistente

8. **Salve o arquivo .pbix**:
   - Salve na pasta `reports/dashboard`

## 9. Conclusão

Neste notebook, implementamos um pipeline completo de ETL para dados de e-commerce, incluindo:

1. **Extração** de dados de múltiplas fontes
2. **Transformação** dos dados brutos em um modelo dimensional (estrela)
3. **Criação** de tabelas agregadas para análise
4. **Carga** dos dados transformados em formatos adequados para o Power BI
5. **Verificação** da integridade dos dados
6. **Visualização** preliminar para o dashboard
7. **Instruções** para integração com Power BI

Os dados estão prontos para serem importados no Power BI para criar um dashboard interativo e completo de análise de vendas de e-commerce.